In [8]:
import pathlib
import time
import zipfile
import sys
import shutil  # used for safe file extraction (copyfileobj + atomic rename)
import duckdb
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# -----------------------
# Page element IDs
# -----------------------
YEAR_DROPDOWN_ID = "cboYear"        # <select> with the years
QUARTER_DROPDOWN_ID = "cboPeriod"   # <select> with the quarters (ignored if QUARTERS=[])
DOWNLOAD_BTN_ID  = "btnDownload"    # "Download" button
CHECKBOX_ID      = "chkAllVars"     # "Select all variables" (used if REQUIRED_FIELDS=[])

# -----------------------
# Run settings
# -----------------------
FIRST_YEAR, LAST_YEAR = 2005, 2024      # inclusive loop
TIMEOUT_SEC           = 600             # max wait for page elements
HEADLESS              = True            # set False to watch the browser
DOWNLOAD_WAIT_SEC     = 600             # max time to wait for a download
OVERWRITE_FILES       = "overwrite"     # "overwrite", "timestamp", or "skip"
REFRESH_EVERY_N_DOWNLOADS = 4           # refresh page every N downloads to avoid popups

# -----------------------
# Paths
# -----------------------
DATA_FOLDER = pathlib.Path("/home/akimovh/predatory_pricing_airlines/data")

# Selenium extractor

In [2]:
def make_driver() -> webdriver.Chrome:
    """Create Chrome driver with proper download settings."""
    opts = webdriver.ChromeOptions()

    if HEADLESS:
        opts.add_argument("--headless=new")

    if sys.platform.startswith("linux"):
        opts.add_argument("--no-sandbox")
        opts.add_argument("--disable-dev-shm-usage")
        opts.add_argument("--disable-gpu")
        opts.add_argument("--disable-software-rasterizer")

    prefs = {
        "download.default_directory": str(ZIP_DIR.resolve()),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0,
        "profile.default_content_setting_values.automatic_downloads": 1,
    }
    opts.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(options=opts)

    try:
        driver.execute_cdp_cmd(
            "Page.setDownloadBehavior",
            {"behavior": "allow", "downloadPath": str(ZIP_DIR.resolve())},
        )
    except Exception:
        driver.execute_cdp_cmd(
            "Browser.setDownloadBehavior",
            {"behavior": "allow", "downloadPath": str(ZIP_DIR.resolve())},
        )

    return driver


def wait_for_download_complete(
    initial_files: set, max_wait_sec: int = DOWNLOAD_WAIT_SEC
) -> pathlib.Path | None:
    """Wait for a new file to appear in ZIP_DIR and finish downloading."""
    start_time = time.time()
    last_sizes = {}

    while time.time() - start_time < max_wait_sec:
        if list(ZIP_DIR.glob("*.crdownload")):
            print("Download in progress...")
            time.sleep(2)
            continue

        current_files = set(ZIP_DIR.iterdir())
        new_files = [f for f in (current_files - initial_files) if f.is_file()]
        new_files = [f for f in new_files if f.stat().st_size > 1000]

        if new_files:
            newest = max(new_files, key=lambda p: p.stat().st_mtime)
            size_now = newest.stat().st_size
            size_prev = last_sizes.get(newest, None)
            last_sizes[newest] = size_now

            if size_prev is not None and size_now == size_prev:
                print(f"✓ Download complete: {newest.name} ({size_now} bytes)")
                return newest

        time.sleep(1)

    print(f"⚠ Download timeout after {max_wait_sec} seconds")
    return None


def dismiss_popups(driver, wait):
    """Try to dismiss any survey popups or overlays that might be blocking the page."""
    try:
        popup_selectors = [
            "div[id*='QSIPopOver']",
            "div[class*='popup']",
            "div[class*='overlay']",
            "div[class*='modal']",
            "button[id*='close']",
            "button[class*='close']",
            "[aria-label*='close']",
            "[aria-label*='Close']",
        ]

        dismissed = False
        for selector in popup_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if element.is_displayed():
                        print(f"    🚫 Dismissing popup: {selector}")
                        element.click()
                        dismissed = True
                        time.sleep(1)
                        break
            except Exception:
                continue

        if not dismissed:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
                time.sleep(1)
            except Exception:
                pass

    except Exception as e:
        print(f"    ⚠ Popup dismissal failed: {e}")


def select_data_fields(driver, wait):
    """Select data fields - either use 'select all' checkbox or individual fields."""
    if not REQUIRED_FIELDS:
        try:
            print("  📋 Using 'Select All' checkbox...")
            chk = wait.until(EC.element_to_be_clickable((By.ID, CHECKBOX_ID)))
            if not chk.is_selected():
                chk.click()
                print("  ✓ All fields selected via checkbox")
            else:
                print("  ✓ All fields already selected")
        except Exception as e:
            print(f"  ⚠ Select all checkbox failed: {e}")
            print("  🔄 Falling back to manual field selection...")
        else:
            try:
                zip_checkbox = driver.find_element(By.ID, "chkDownloadZip")
                if not zip_checkbox.is_selected():
                    zip_checkbox.click()
            except Exception:
                pass
            return True, []

    print(f"  📋 Selecting {len(REQUIRED_FIELDS)} required data fields...")
    selected_count = 0
    missing_fields = []

    for field_name in REQUIRED_FIELDS:
        possible_selectors = [
            f"input[value='{field_name}']",
            f"input[name*='{field_name}']",
            f"input[id*='{field_name}']",
            f"//input[@value='{field_name}']",
            f"//input[contains(@name, '{field_name}')]",
            f"//input[contains(@id, '{field_name}')]",
        ]

        field_found = False
        for selector in possible_selectors:
            try:
                element = (
                    driver.find_element(By.XPATH, selector)
                    if selector.startswith("//")
                    else driver.find_element(By.CSS_SELECTOR, selector)
                )
                if not element.is_selected():
                    driver.execute_script("arguments[0].scrollIntoView(true);", element)
                    time.sleep(0.1)
                    element.click()
                selected_count += 1
                field_found = True
                print(f"    ✓ {field_name}")
                break
            except Exception:
                continue

        if not field_found:
            missing_fields.append(field_name)
            print(f"    ❌ {field_name} (not found)")

    if selected_count > 0:
        try:
            zip_checkbox = driver.find_element(By.ID, "chkDownloadZip")
            if not zip_checkbox.is_selected():
                zip_checkbox.click()
        except Exception:
            pass

    return selected_count > 0, missing_fields


def click_download_button_robustly(driver, wait):
    """Try multiple methods to click the download button, handling popups and overlays."""
    try:
        dismiss_popups(driver, wait)
        dl_btn = wait.until(EC.element_to_be_clickable((By.ID, DOWNLOAD_BTN_ID)))
        dl_btn.click()
        print("    ✓ Download button clicked (normal)")
        return True
    except Exception as e:
        print(f"    ⚠ Normal click failed: {str(e)[:100]}...")

    try:
        dl_btn = driver.find_element(By.ID, DOWNLOAD_BTN_ID)
        driver.execute_script("arguments[0].click();", dl_btn)
        print("    ✓ Download button clicked (JavaScript)")
        return True
    except Exception as e:
        print(f"    ⚠ JavaScript click failed: {str(e)[:100]}...")

    try:
        dl_btn = driver.find_element(By.ID, DOWNLOAD_BTN_ID)
        driver.execute_script("arguments[0].scrollIntoView(true);", dl_btn)
        time.sleep(1)
        dl_btn.click()
        print("    ✓ Download button clicked (scroll + click)")
        return True
    except Exception as e:
        print(f"    ⚠ Scroll + click failed: {str(e)[:100]}...")

    raise Exception("All download button click methods failed")


def is_valid_zip(file_path: pathlib.Path) -> bool:
    """Check if a file is a valid ZIP archive."""
    try:
        with zipfile.ZipFile(file_path, "r") as z:
            z.namelist()
        return True
    except zipfile.BadZipFile:
        return False
    except Exception as e:
        print(f"Error checking ZIP file: {e}")
        return False


def unzip_to_raw(
    year: int, quarter: int = None, zip_path: pathlib.Path = None, overwrite: str = "skip"
) -> bool:
    """Extract ZIP file to RAW_DIR with prefixed filenames (flat, no subfolders)."""
    try:
        RAW_DIR.mkdir(parents=True, exist_ok=True)
        prefix = f"{year}_Q{quarter}" if quarter is not None else f"{year}"

        with zipfile.ZipFile(zip_path, "r") as z:
            members = [m for m in z.infolist() if not m.is_dir()]
            print(f"  Extracting {len(members)} files with prefix {prefix}")

            extracted_count = 0
            skipped_count = 0

            for m in members:
                base_name = pathlib.Path(m.filename).name
                stem = pathlib.Path(base_name).stem
                suffix = pathlib.Path(base_name).suffix

                new_filename = f"{prefix}_{stem}{suffix}"
                out_path = RAW_DIR / new_filename

                if out_path.exists():
                    if overwrite == "overwrite":
                        out_path.unlink()
                    elif overwrite == "timestamp":
                        import datetime
                        timestamp = datetime.datetime.now().strftime("%H%M%S")
                        out_path = RAW_DIR / f"{prefix}_{stem}_{timestamp}{suffix}"
                    elif overwrite == "skip":
                        skipped_count += 1
                        continue

                tmp_path = out_path.with_suffix(out_path.suffix + ".tmp")
                with z.open(m, "r") as src, open(tmp_path, "wb") as dst:
                    shutil.copyfileobj(src, dst)
                tmp_path.replace(out_path)

                extracted_count += 1

            print(f"  📊 Extracted: {extracted_count}, Skipped: {skipped_count}")
        return True

    except Exception as e:
        print(f"Error extracting {zip_path}: {e}")
        return False


def main():
    driver = None
    downloads_processed = 0
    is_quarterly = len(QUARTERS) > 0
    data_type = "quarterly" if is_quarterly else "annual"

    try:
        print(f"Starting BTS {data_type} data download...")
        print("Dataset: DB1B (Quarterly)" if is_quarterly else "Dataset: T100D (Annual)")

        driver = make_driver()
        wait = WebDriverWait(driver, TIMEOUT_SEC)

        print(f"Navigating to: {START_URL}")
        driver.get(START_URL)

        try:
            wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
            print("✓ Page loaded successfully")
        except TimeoutException:
            print("❌ Page failed to load properly")
            return

        quarters_to_process = QUARTERS if is_quarterly else [None]

        for yr in range(FIRST_YEAR, LAST_YEAR + 1):
            for qtr in quarters_to_process:
                download_id = f"{yr}_Q{qtr}" if qtr is not None else f"{yr}"
                print(f"\n📥 Processing {download_id}...")

                if downloads_processed > 0 and downloads_processed % REFRESH_EVERY_N_DOWNLOADS == 0:
                    print("  🔄 Refreshing page to avoid popups...")
                    driver.get(START_URL)
                    try:
                        wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
                        print("  ✓ Page refreshed successfully")
                    except TimeoutException:
                        print("  ❌ Page refresh failed")
                        continue

                initial_files = set(ZIP_DIR.iterdir())

                try:
                    print(f"  Selecting year {yr}")
                    year_dropdown = Select(driver.find_element(By.ID, YEAR_DROPDOWN_ID))
                    year_dropdown.select_by_visible_text(str(yr))
                    time.sleep(1)

                    if qtr is not None:
                        print(f"  Selecting quarter {qtr}")
                        quarter_dropdown = Select(driver.find_element(By.ID, QUARTER_DROPDOWN_ID))
                        quarter_dropdown.select_by_visible_text(f"Quarter {qtr}")
                        time.sleep(1)

                    fields_selected, _ = select_data_fields(driver, wait)
                    if not fields_selected:
                        print(f"❌ No fields selected for {download_id}")
                        continue

                    print("  Clicking download button")
                    click_download_button_robustly(driver, wait)

                    print("  Waiting for download to complete...")
                    downloaded_file = wait_for_download_complete(initial_files)
                    if downloaded_file is None:
                        print(f"❌ Download failed for {download_id}")
                        continue

                    if not is_valid_zip(downloaded_file):
                        print(f"❌ Downloaded file is not a valid ZIP: {downloaded_file}")
                        try:
                            downloaded_file.unlink()
                        except Exception:
                            pass
                        continue

                    # Rename locally to a clean name
                    clean_name = ZIP_DIR / f"{download_id}.zip"
                    if clean_name.exists():
                        clean_name.unlink()
                    downloaded_file.rename(clean_name)
                    print(f"  Renamed to: {clean_name.name}")

                    # Unzip locally
                    print("  Extracting files...")
                    if unzip_to_raw(yr, qtr, clean_name, overwrite=OVERWRITE_FILES):
                        print(f"✅ {download_id} completed successfully")
                        downloads_processed += 1

                        if DELETE_ZIP_AFTER_EXTRACT:
                            try:
                                clean_name.unlink()
                                print(f"  🧹 Deleted ZIP after extract: {clean_name.name}")
                            except Exception as e:
                                print(f"  ⚠ Could not delete ZIP {clean_name.name}: {e}")
                    else:
                        print(f"❌ Extraction failed for {download_id}")
                        try:
                            if clean_name.exists():
                                clean_name.unlink()
                        except Exception:
                            pass

                except TimeoutException as e:
                    print(f"❌ Timeout error for {download_id}: {e}")
                    print("  🔄 Attempting page refresh due to timeout...")
                    try:
                        driver.get(START_URL)
                        wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
                    except Exception:
                        print("  ❌ Recovery refresh failed")

                except Exception as e:
                    print(f"❌ Error processing {download_id}: {e}")
                    if "click intercepted" in str(e) or "popup" in str(e).lower():
                        print("  🔄 Popup detected, refreshing page...")
                        try:
                            driver.get(START_URL)
                            wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
                        except Exception:
                            print("  ❌ Recovery refresh failed")

    except Exception as e:
        print(f"❌ Fatal error: {e}")

    finally:
        if driver:
            driver.quit()
            print("\n🔒 Browser closed")

    print(f"\n📋 SUMMARY:")
    print(f"  Data type: {data_type}")
    print(f"  Downloads processed: {downloads_processed}")
    print(f"  ZIP dir: {ZIP_DIR}")
    print(f"  RAW dir: {RAW_DIR}")

# T100

In [3]:
# -----------------------
# CONFIG: T100D (Annual) — CLUSTER LOCAL ONLY
# -----------------------

DATASET_ROOT = DATA_FOLDER / "T100"
ZIP_DIR = DATASET_ROOT / "zip"
RAW_DIR = DATASET_ROOT / "raw"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)

# If True: delete each yearly zip after a successful extract
DELETE_ZIP_AFTER_EXTRACT = False

START_URL = "https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=GEE&QO_fu146_anzr=Nv4+Pn44vr45"
QUARTERS = []          # Empty = annual data
REQUIRED_FIELDS = []   # Empty = use "select all" button
CHECKBOX_ID = "chkAllVars"


def cleanup_local_folders():
    """
    Safety cleanup: remove leftover partial downloads or temp files.
    """
    for p in ZIP_DIR.glob("*.crdownload"):
        try:
            p.unlink()
        except Exception:
            pass

    for p in RAW_DIR.glob("*.tmp"):
        try:
            p.unlink()
        except Exception:
            pass


try:
    main()
    cleanup_local_folders()

    print("\n✅ Done.")
    print(f"   ZIPs → {ZIP_DIR}")
    print(f"   Extracted files → {RAW_DIR}")

except KeyboardInterrupt:
    print("\n⏹ Download interrupted by user")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")

except Exception as e:
    print(f"\n💥 Unexpected error: {e}")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")
    sys.exit(1)

Starting BTS annual data download...
Dataset: T100D (Annual)
Navigating to: https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=GEE&QO_fu146_anzr=Nv4+Pn44vr45
✓ Page loaded successfully

📥 Processing 2005...
  Selecting year 2005
  📋 Using 'Select All' checkbox...
  ✓ All fields selected via checkbox
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
Download in progress...
Download in progress...
Download in progress...
Download in progress...
Download in progress...
✓ Download complete: T_T100D_SEGMENT_ALL_CARRIER_20260115_111648.zip (12043356 bytes)
  Renamed to: 2005.zip
  Extracting files...
  Extracting 1 files with prefix 2005
  📊 Extracted: 1, Skipped: 0
✅ 2005 completed successfully

📥 Processing 2006...
  Selecting year 2006
  📋 Using 'Select All' checkbox...
  ✓ All fields already selected
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
Download in progress...

# DB1B

## Market

In [8]:
# -----------------------
# CONFIG: DB1B (Quarterly) — CLUSTER LOCAL ONLY
# -----------------------

DATASET_ROOT = DATA_FOLDER / "DB1B" / "Market"
ZIP_DIR = DATASET_ROOT / "zip"
RAW_DIR = DATASET_ROOT / "raw"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)

# If True: delete each quarterly zip after a successful extract
DELETE_ZIP_AFTER_EXTRACT = False

START_URL = "https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FHK&QO_fu146_anzr=b4vtv0+n0q+Qr56v0n6v10+f748rB"
QUARTERS = [1, 2, 3, 4]  # quarterly
REQUIRED_FIELDS = [
    "ITIN_ID", "MKT_ID", "MARKET_COUPONS", "YEAR", "QUARTER", "ORIGIN", "ORIGIN_COUNTRY",
    "ORIGIN_STATE_FIPS", "ORIGIN_STATE_ABR", "ORIGIN_STATE_NM", "ORIGIN_WAC", "DEST",
    "DEST_COUNTRY", "DEST_STATE_FIPS", "DEST_STATE_ABR", "DEST_STATE_NM", "DEST_WAC",
    "AIRPORT_GROUP", "WAC_GROUP", "TK_CARRIER_CHANGE", "TK_CARRIER_GROUP",
    "OP_CARRIER_CHANGE", "OP_CARRIER_GROUP", "REPORTING_CARRIER", "TICKET_CARRIER",
    "OPERATING_CARRIER", "BULK_FARE", "PASSENGERS", "MARKET_FARE", "MARKET_DISTANCE",
    "DISTANCE_GROUP", "MARKET_MILES_FLOWN", "NONSTOP_MILES", "ITIN_GEO_TYPE", "MKT_GEO_TYPE"
]
CHECKBOX_ID = "chkAllVars"  # unused when REQUIRED_FIELDS is non-empty, but safe to keep


def cleanup_local_folders():
    """Remove leftovers if interrupted mid-download."""
    for p in ZIP_DIR.glob("*.crdownload"):
        try:
            p.unlink()
        except Exception:
            pass
    for p in RAW_DIR.glob("*.tmp"):
        try:
            p.unlink()
        except Exception:
            pass


try:
    main()
    cleanup_local_folders()

    print("\n✅ Done.")
    print(f"   ZIPs → {ZIP_DIR}")
    print(f"   Extracted files → {RAW_DIR}")

except KeyboardInterrupt:
    print("\n⏹ Download interrupted by user")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")

except Exception as e:
    print(f"\n💥 Unexpected error: {e}")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")
    sys.exit(1)

Starting BTS quarterly data download...
Dataset: DB1B (Quarterly)
Navigating to: https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FHK&QO_fu146_anzr=b4vtv0+n0q+Qr56v0n6v10+f748rB
✓ Page loaded successfully

📥 Processing 2014_Q1...
  Selecting year 2014
  Selecting quarter 1
  📋 Selecting 35 required data fields...
    ✓ ITIN_ID
    ✓ MKT_ID
    ✓ MARKET_COUPONS
    ✓ YEAR
    ✓ QUARTER
    ✓ ORIGIN
    ✓ ORIGIN_COUNTRY
    ✓ ORIGIN_STATE_FIPS
    ✓ ORIGIN_STATE_ABR
    ✓ ORIGIN_STATE_NM
    ✓ ORIGIN_WAC
    ✓ DEST
    ✓ DEST_COUNTRY
    ✓ DEST_STATE_FIPS
    ✓ DEST_STATE_ABR
    ✓ DEST_STATE_NM
    ✓ DEST_WAC
    ✓ AIRPORT_GROUP
    ✓ WAC_GROUP
    ✓ TK_CARRIER_CHANGE
    ✓ TK_CARRIER_GROUP
    ✓ OP_CARRIER_CHANGE
    ✓ OP_CARRIER_GROUP
    ✓ REPORTING_CARRIER
    ✓ TICKET_CARRIER
    ✓ OPERATING_CARRIER
    ✓ BULK_FARE
    ✓ PASSENGERS
    ✓ MARKET_FARE
    ✓ MARKET_DISTANCE
    ✓ DISTANCE_GROUP
    ✓ MARKET_MILES_FLOWN
    ✓ NONSTOP_MILES
    ✓ ITIN_GEO_TYPE
    ✓ MKT_GEO_T

## Ticket

In [14]:
# -----------------------
# CONFIG: DB1B (Quarterly) — CLUSTER LOCAL ONLY
# -----------------------

DATASET_ROOT = DATA_FOLDER / "DB1B" / "Ticket"
ZIP_DIR = DATASET_ROOT / "zip"
RAW_DIR = DATASET_ROOT / "raw"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)

# If True: delete each quarterly zip after a successful extract
DELETE_ZIP_AFTER_EXTRACT = False

START_URL = "https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FKF&QO_fu146_anzr=b4vtv0%20n0q%20Qr56v0n6v10%20f748rB"
QUARTERS = [1, 2, 3, 4]  # quarterly
REQUIRED_FIELDS = ["ITIN_ID", "COUPONS", "YEAR", "QUARTER", "ORIGIN", "ORIGIN_AIRPORT_ID", "ORIGIN_AIRPORT_SEQ_ID",
                   "ORIGIN_CITY_MARKET_ID", "ORIGIN_COUNTRY", "ORIGIN_STATE_FIPS", "ORIGIN_STATE_ABR", "ORIGIN_STATE_NM",
                   "ORIGIN_WAC", "ROUNDTRIP", "ONLINE", "DOLLAR_CRED", "ITIN_YIELD", "REPORTING_CARRIER", "PASSENGERS", 
                   "ITIN_FARE", "BULK_FARE", "DISTANCE", "DISTANCE_GROUP", "MILES_FLOWN", "ITIN_GEO_TYPE"]

CHECKBOX_ID = "chkAllVars"  # unused when REQUIRED_FIELDS is non-empty, but safe to keep


def cleanup_local_folders():
    """Remove leftovers if interrupted mid-download."""
    for p in ZIP_DIR.glob("*.crdownload"):
        try:
            p.unlink()
        except Exception:
            pass
    for p in RAW_DIR.glob("*.tmp"):
        try:
            p.unlink()
        except Exception:
            pass


try:
    main()
    cleanup_local_folders()

    print("\n✅ Done.")
    print(f"   ZIPs → {ZIP_DIR}")
    print(f"   Extracted files → {RAW_DIR}")

except KeyboardInterrupt:
    print("\n⏹ Download interrupted by user")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")

except Exception as e:
    print(f"\n💥 Unexpected error: {e}")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")
    sys.exit(1)


Starting BTS quarterly data download...
Dataset: DB1B (Quarterly)
Navigating to: https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FKF&QO_fu146_anzr=b4vtv0%20n0q%20Qr56v0n6v10%20f748rB
✓ Page loaded successfully

📥 Processing 2019_Q1...
  Selecting year 2019
  Selecting quarter 1
  📋 Selecting 25 required data fields...
    ✓ ITIN_ID
    ✓ COUPONS
    ✓ YEAR
    ✓ QUARTER
    ✓ ORIGIN
    ✓ ORIGIN_AIRPORT_ID
    ✓ ORIGIN_AIRPORT_SEQ_ID
    ✓ ORIGIN_CITY_MARKET_ID
    ✓ ORIGIN_COUNTRY
    ✓ ORIGIN_STATE_FIPS
    ✓ ORIGIN_STATE_ABR
    ✓ ORIGIN_STATE_NM
    ✓ ORIGIN_WAC
    ✓ ROUNDTRIP
    ✓ ONLINE
    ✓ DOLLAR_CRED
    ✓ ITIN_YIELD
    ✓ REPORTING_CARRIER
    ✓ PASSENGERS
    ✓ ITIN_FARE
    ✓ BULK_FARE
    ✓ DISTANCE
    ✓ DISTANCE_GROUP
    ✓ MILES_FLOWN
    ✓ ITIN_GEO_TYPE
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
Download in progress...
Download in progress...
Download in progress...
Download in progress...
Down

# T-F41

In [6]:
# -----------------------
# CONFIG: T-F41 (Schedule P-1.2) — CLUSTER LOCAL ONLY
# -----------------------

FIRST_YEAR, LAST_YEAR = 2005, 2024

DATASET_ROOT = DATA_FOLDER / "T-F41"
ZIP_DIR = DATASET_ROOT / "zip"
RAW_DIR = DATASET_ROOT / "raw"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)

# If True: delete each zip after a successful extract
DELETE_ZIP_AFTER_EXTRACT = False

# Air Carrier Financial : Schedule P-1.2
START_URL = "https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FMI&QO_fu146_anzr=Nv4%20Pn44vr4%20Sv0n0pvny"

# Annual (no quarters)
QUARTERS = []
REQUIRED_FIELDS = []        # empty => select-all behavior
CHECKBOX_ID = "chkAllVars"  # keep for pages where it’s a checkbox


def cleanup_local_folders():
    """Remove leftovers if interrupted mid-download."""
    for p in ZIP_DIR.glob("*.crdownload"):
        try:
            p.unlink()
        except Exception:
            pass
    for p in RAW_DIR.glob("*.tmp"):
        try:
            p.unlink()
        except Exception:
            pass


try:
    main()
    cleanup_local_folders()

    print("\n✅ Done.")
    print(f"   ZIPs → {ZIP_DIR}")
    print(f"   Extracted files → {RAW_DIR}")

except KeyboardInterrupt:
    print("\n⏹ Download interrupted by user")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")

except Exception as e:
    print(f"\n💥 Unexpected error: {e}")
    cleanup_local_folders()
    print("🧹 Cleaned temp leftovers")
    sys.exit(1)


Starting BTS annual data download...
Dataset: T100D (Annual)
Navigating to: https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FMI&QO_fu146_anzr=Nv4%20Pn44vr4%20Sv0n0pvny
✓ Page loaded successfully

📥 Processing 2005...
  Selecting year 2005
  📋 Using 'Select All' checkbox...
  ✓ All fields selected via checkbox
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
✓ Download complete: T_F41SCHEDULE_P12_20260115_140547.zip (48664 bytes)
  Renamed to: 2005.zip
  Extracting files...
  Extracting 1 files with prefix 2005
  📊 Extracted: 1, Skipped: 0
✅ 2005 completed successfully

📥 Processing 2006...
  Selecting year 2006
  📋 Using 'Select All' checkbox...
  ✓ All fields already selected
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
✓ Download complete: T_F41SCHEDULE_P12_20260115_140553.zip (49629 bytes)
  Renamed to: 2006.zip
  Extracting files...
  Extracting 1 files with

  ✓ Page refreshed successfully
  Selecting year 2013
  📋 Using 'Select All' checkbox...
  ✓ All fields selected via checkbox
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
✓ Download complete: T_F41SCHEDULE_P12_20260115_140643.zip (43495 bytes)
  Renamed to: 2013.zip
  Extracting files...
  Extracting 1 files with prefix 2013
  📊 Extracted: 1, Skipped: 0
✅ 2013 completed successfully

📥 Processing 2014...
  Selecting year 2014
  📋 Using 'Select All' checkbox...
  ✓ All fields already selected
  Clicking download button
    ✓ Download button clicked (normal)
  Waiting for download to complete...
✓ Download complete: T_F41SCHEDULE_P12_20260115_140648.zip (41906 bytes)
  Renamed to: 2014.zip
  Extracting files...
  Extracting 1 files with prefix 2014
  📊 Extracted: 1, Skipped: 0
✅ 2014 completed successfully

📥 Processing 2015...
  Selecting year 2015
  📋 Using 'Select All' checkbox...
  ✓ All fields already selected
  Clicking dow

# Create one duckdb database

In [13]:
con = duckdb.connect(DATA_FOLDER / "duckdb" / "airlines.duckdb")
con.execute(f"PRAGMA threads={os.cpu_count()}")

T100

In [14]:
folder = DATA_FOLDER / "T100/raw"
t100_files = [str(p) for p in folder.iterdir() if p.is_file()]
con.execute("""
                create or replace table T100 as
                select *
                from read_csv_auto($files)
            """, {"files": t100_files})

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

DB1B Market

In [4]:
# -----------------------
# DB1B Market (local CSV -> DuckDB table)
# -----------------------
folder  = DATA_FOLDER / "DB1B" / "Market" / "raw"
pattern = str(folder / "*.csv")  # DuckDB glob

schema = {
    "ItinID": "BIGINT",
    "MktID": "BIGINT",
    "MktCoupons": "BIGINT",
    "Year": "BIGINT",
    "Quarter": "BIGINT",

    "OriginAirportID": "BIGINT",
    "OriginAirportSeqID": "BIGINT",
    "OriginCityMarketID": "BIGINT",
    "Origin": "VARCHAR",
    "OriginCountry": "VARCHAR",
    "OriginStateFips": "BIGINT",
    "OriginState": "VARCHAR",
    "OriginStateName": "VARCHAR",
    "OriginWac": "BIGINT",

    "DestAirportID": "BIGINT",
    "DestAirportSeqID": "BIGINT",
    "DestCityMarketID": "BIGINT",
    "Dest": "VARCHAR",
    "DestCountry": "VARCHAR",
    "DestStateFips": "BIGINT",
    "DestState": "VARCHAR",
    "DestStateName": "VARCHAR",
    "DestWac": "BIGINT",

    "AirportGroup": "VARCHAR",
    "WacGroup": "VARCHAR",

    "TkCarrierChange": "DOUBLE",
    "TkCarrierGroup": "VARCHAR",
    "OpCarrierChange": "DOUBLE",
    "OpCarrierGroup": "VARCHAR",

    "RPCarrier": "VARCHAR",
    "TkCarrier": "VARCHAR",
    "OpCarrier": "VARCHAR",

    "BulkFare": "DOUBLE",
    "Passengers": "DOUBLE",
    "MktFare": "DOUBLE",
    "MktDistance": "DOUBLE",
    "MktDistanceGroup": "BIGINT",
    "MktMilesFlown": "DOUBLE",
    "NonStopMiles": "DOUBLE",

    "ItinGeoType": "BIGINT",
    "MktGeoType": "BIGINT",

    "Unnamed: 41": "DOUBLE",
}

def q(col: str) -> str:
    # Quote everything (handles spaces/colons safely, e.g., "Unnamed: 41")
    return f'"{col.replace(chr(34), chr(34)*2)}"'

select_list = ",\n        ".join(q(c) for c in schema.keys())

con.execute(
    f"""
    create or replace table DB1B_Market as
    select
        {select_list}
    FROM read_csv(
        ?,                      -- glob pattern
        header = true,
        columns = ?,            -- schema dict (stable types)
        parallel = true,
        union_by_name = true,   -- tolerant to minor column-order/layout differences
        strict_mode = false,
        ignore_errors = true,
        normalize_names = false -- keep exact names like "Unnamed: 41"
    )
    """,
    [pattern, schema],
)

rows = con.execute("select count(*) from DB1B_Market").fetchone()[0]
print(f"✅ DB1B_Market load complete ({rows:,} rows)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ DB1B_Market load complete (469,051,255 rows)


DB1B Ticket

In [5]:
# -----------------------
# DB1B Ticket (local CSV -> DuckDB table)
# -----------------------
folder  = DATA_FOLDER / "DB1B" / "Ticket" / "raw"
pattern = str(folder / "*.csv")  # DuckDB glob

if not folder.exists():
    raise FileNotFoundError(f"Folder not found: {folder}")
if not any(folder.glob("*.csv")):
    raise FileNotFoundError(f"No .csv files found in: {folder}")

schema = {
    "ItinID": "BIGINT",
    "Coupons": "BIGINT",
    "Year": "BIGINT",
    "Quarter": "BIGINT",

    "Origin": "VARCHAR",
    "OriginAirportID": "BIGINT",
    "OriginAirportSeqID": "BIGINT",
    "OriginCityMarketID": "BIGINT",
    "OriginCountry": "VARCHAR",
    "OriginStateFips": "BIGINT",
    "OriginState": "VARCHAR",
    "OriginStateName": "VARCHAR",
    "OriginWac": "BIGINT",

    "RoundTrip": "DOUBLE",
    "OnLine": "DOUBLE",
    "DollarCred": "BIGINT",
    "FarePerMile": "DOUBLE",

    "RPCarrier": "VARCHAR",
    "Passengers": "DOUBLE",
    "ItinFare": "DOUBLE",
    "BulkFare": "DOUBLE",

    "Distance": "DOUBLE",
    "DistanceGroup": "BIGINT",
    "MilesFlown": "DOUBLE",

    "ItinGeoType": "BIGINT",

    "Unnamed: 25": "DOUBLE",
}

def q(col: str) -> str:
    return f'"{col.replace(chr(34), chr(34)*2)}"'

select_list = ",\n        ".join(q(c) for c in schema.keys())

con.execute(
    f"""
    CREATE OR REPLACE TABLE DB1B_Ticket AS
    SELECT
        {select_list}
    FROM read_csv(
        ?,                      -- glob pattern
        header = true,
        columns = ?,            -- schema dict (stable types)
        parallel = true,
        union_by_name = true,
        strict_mode = false,
        ignore_errors = true,
        normalize_names = false -- keep exact names like "Unnamed: 25"
    )
    """,
    [pattern, schema],
)

rows = con.execute("SELECT COUNT(*) FROM DB1B_Ticket").fetchone()[0]
print(f"✅ DB1B_Ticket load complete ({rows:,} rows)")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ DB1B_Ticket load complete (280,724,306 rows)


T-F41

In [6]:
folder = DATA_FOLDER / "T-F41" / "raw"

T_F41_files = [str(p) for p in folder.iterdir() if p.is_file() and p.suffix.lower() == ".csv"]

con.execute("""
                CREATE OR REPLACE TABLE T_F41 AS
                SELECT *
                FROM read_csv_auto($files)
            """, {"files": T_F41_files})

In [7]:
con.close()